In [1]:
# you can ignore this step if you ran pip install requirements while setting up the python virtual env
!pip install openai pymongo numpy pandas s3fs langchain-mongodb langchain langchain-community langchain-openai python-dotenv

  Obtaining dependency information for botocore<1.34.132,>=1.34.70 from https://files.pythonhosted.org/packages/46/1a/01785fad12a9b1dbeffebd97cd226ea5923114057c64a610dd4eb8a28c7b/botocore-1.34.131-py3-none-any.whl.metadata
  Using cached botocore-1.34.131-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.34.131-py3-none-any.whl (12.3 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.35.2
    Uninstalling botocore-1.35.2:
      Successfully uninstalled botocore-1.35.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.35.2 requires botocore<1.36.0,>=1.35.2, but you have botocore 1.34.131 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [15]:
import openai
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass("Input Open AI API KEY:")

if 'MONGODB_CONNECTION_STR' not in os.environ:
    os.environ['MONGODB_CONNECTION_STRING'] = getpass("Input MongoDB Connection String:")

response = openai.embeddings.create(
  input="Educative answers section is helpful",
  model="text-embedding-ada-002"
)

In [26]:
import pandas as pd
import s3fs
s3_uri = "s3://ashwin-partner-bucket/movies_sample_dataset.jsonl"
df = pd.read_json(s3_uri, orient="records", lines=True)
df.to_json("./movies_sample_dataset.jsonl", orient="records", lines=True)
df[:3]

,overview,title,release_date,vote_average,vote_count,adult,year,month,day,text
0,"Led by Woody, Andy's toys live happily in his ...",Toy Story,1995-10-30,7.7,5415,False,1995,10,30,"Title: Toy Story Genres: Animation,Comedy,Fam..."
1,When siblings Judy and Peter discover an encha...,Jumanji,1995-12-15,6.9,2413,False,1995,12,15,"Title: Jumanji Genres: Animation,Comedy,Famil..."
2,A family wedding reignites the ancient feud be...,Grumpier Old Men,1995-12-22,6.5,92,False,1995,12,22,"Title: Grumpier Old Men Genres: Animation,Com..."


In [146]:
import numpy as np
from tqdm import tqdm
df['final'] = df['text'] + "    Overview: " + df['overview']
df['final'][:5]
step =int(np.ceil(df['final'].shape[0]/100))
embeddings_t = []
lines = []
for x,y in list(map(lambda x:(x,x+step), list(range(0,df.shape[0],step)))):
    lines += [df.final.values[x:y].tolist()]
for i in tqdm(lines):
    embeddings_t += openai.embeddings.create(model='text-embedding-ada-002', input=i).data
    
out = []
for ele in embeddings_t:
    out += [ele.embedding]
df['embedding'] = out
df[:5]

100%|██████████| 100/100 [07:03<00:00,  4.24s/it]


In [153]:
df[:5]

,overview,title,release_date,vote_average,vote_count,adult,year,month,day,text,final,embedding
0,"Led by Woody, Andy's toys live happily in his ...",Toy Story,1995-10-30,7.7,5415,False,1995,10,30,"Title: Toy Story Genres: Animation,Comedy,Fam...","Title: Toy Story Genres: Animation,Comedy,Fam...","[-0.012458283454179764, -0.042695507407188416,..."
1,When siblings Judy and Peter discover an encha...,Jumanji,1995-12-15,6.9,2413,False,1995,12,15,"Title: Jumanji Genres: Animation,Comedy,Famil...","Title: Jumanji Genres: Animation,Comedy,Famil...","[0.015389477834105492, -0.028312528505921364, ..."
2,A family wedding reignites the ancient feud be...,Grumpier Old Men,1995-12-22,6.5,92,False,1995,12,22,"Title: Grumpier Old Men Genres: Animation,Com...","Title: Grumpier Old Men Genres: Animation,Com...","[0.016336416825652122, -0.01960906572639942, 0..."
3,"Cheated on, mistreated and stepped on, the wom...",Waiting to Exhale,1995-12-22,6.1,34,False,1995,12,22,"Title: Waiting to Exhale Genres: Animation,Co...","Title: Waiting to Exhale Genres: Animation,Co...","[-0.018749399110674858, -0.0272560715675354, 0..."
4,Just when George Banks has recovered from his ...,Father of the Bride Part II,1995-02-10,5.7,173,False,1995,2,10,Title: Father of the Bride Part II Genres: An...,Title: Father of the Bride Part II Genres: An...,"[0.00420904066413641, -0.020869411528110504, -..."


In [9]:
from pymongo import MongoClient
import certifi
mongo_client = MongoClient(os.environ["MONGODB_CONNECTION_STR"], tlsCAFile=certifi.where())
# Upload documents along with vector embeddings to MongoDB Atlas Collection
output_collection = mongo_client["sample_movies"]["embed_movies"]

In [ ]:
# remove any pre loaded data
if output_collection.count_documents({})>0:
    output_collection.delete_many({})
e = output_collection.insert_many(df.to_dict("records"))

In [10]:
list(output_collection.list_search_indexes())

[{'id': '6627d794b8d9f774f4b2619d',
  'name': 'default',
  'type': 'vectorSearch',
  'status': 'READY',
  'queryable': True,
  'latestDefinitionVersion': {'version': 0,
   'createdAt': datetime.datetime(2024, 4, 23, 15, 45, 24, 356000)},
  'latestDefinition': {'fields': [{'type': 'vector',
     'numDimensions': 1536,
     'path': 'embedding',
     'similarity': 'cosine'},
    {'type': 'filter', 'path': 'year'}]},
  'statusDetail': [{'hostname': 'atlas-uuvcjy-shard-00-01',
    'status': 'READY',
    'queryable': True,
    'mainIndex': {'status': 'READY',
     'queryable': True,
     'definitionVersion': {'version': 0,
      'createdAt': datetime.datetime(2024, 4, 23, 15, 45, 24)},
     'definition': {'fields': [{'type': 'vector',
        'path': 'embedding',
        'numDimensions': 1536,
        'similarity': 'cosine'},
       {'type': 'filter', 'path': 'year'}]}}},
   {'hostname': 'atlas-uuvcjy-shard-00-02',
    'status': 'READY',
    'queryable': True,
    'mainIndex': {'status': 'RE

# Semantic Similarity Search Example

In [199]:
def query_vector_search(q, prefilter = {}, postfilter = {},path="embedding",topK=2):
    ele = openai.embeddings.create(model='text-embedding-ada-002', input=q).data
    query_embedding = ele[0].embedding
    vs_query = {
                "index": "default",
                "path": path,
                "queryVector": query_embedding,
                "numCandidates": 10,
                "limit": topK,
            }
    if len(prefilter)>0:
        vs_query["filter"] = prefilter
    new_search_query = {"$vectorSearch": vs_query}
    project = {"$project": {"score": {"$meta": "vectorSearchScore"},"_id": 0,"title": 1, "release_date": 1, "final": 1,"year": 1}}
    if len(postfilter.keys())>0:
        postFilter = {"$match":postfilter}
        res = list(output_collection.aggregate([new_search_query, project, postFilter]))
    else:
        res = list(output_collection.aggregate([new_search_query, project]))
    return res

In [288]:
query_vector_search("I like Christmas movies, any recommendations for movies release after 1990?", prefilter={"year": {"$gt": 1990}}, topK=5)

[{'title': "Christmas Vacation '91",
  'release_date': '1991-12-20',
  'year': 1991,
  'final': "Title: Christmas Vacation '91  Genres: Animation,Comedy,FamilyThis coarse bedroom farce takes place at the St. Moritz ski resort over a Christmas vacation. Among the couples whose lives intersect are a widowed artist honeymooning with his second wife, a gay man traveling with his son and his lover (and hiding each from the other), a snobbish couple from Milan who have been forced to share a suite with a pair of crass Romans, etc.    Overview: This coarse bedroom farce takes place at the St. Moritz ski resort over a Christmas vacation. Among the couples whose lives intersect are a widowed artist honeymooning with his second wife, a gay man traveling with his son and his lover (and hiding each from the other), a snobbish couple from Milan who have been forced to share a suite with a pair of crass Romans, etc.",
  'score': 0.9068390727043152},
 {'title': 'Happy Christmas',
  'release_date': '2

In [289]:
query_vector_search("I like Christmas movies, any recommendations for movies release after 1990?", prefilter={"year":{"$gt": 1990}}, postfilter={"score": {"$gt":0.905}},topK=5)

[{'title': "Christmas Vacation '91",
  'release_date': '1991-12-20',
  'year': 1991,
  'final': "Title: Christmas Vacation '91  Genres: Animation,Comedy,FamilyThis coarse bedroom farce takes place at the St. Moritz ski resort over a Christmas vacation. Among the couples whose lives intersect are a widowed artist honeymooning with his second wife, a gay man traveling with his son and his lover (and hiding each from the other), a snobbish couple from Milan who have been forced to share a suite with a pair of crass Romans, etc.    Overview: This coarse bedroom farce takes place at the St. Moritz ski resort over a Christmas vacation. Among the couples whose lives intersect are a widowed artist honeymooning with his second wife, a gay man traveling with his son and his lover (and hiding each from the other), a snobbish couple from Milan who have been forced to share a suite with a pair of crass Romans, etc.",
  'score': 0.9068360328674316},
 {'title': 'Happy Christmas',
  'release_date': '2

# Simple/Naive RAG

In [237]:
from openai import OpenAI
client = OpenAI()
def invoke_llm(prompt, model_name='gpt-3.5-turbo-0125'):
    """
    Queries with input prompt to OpenAI API using the chat completion api gets the model's response.
    """
    
    response = client.chat.completions.create(
      model=model_name,
      messages=[
        {
          "role": "user",
          "content": prompt
        }
      ],
      temperature=0.2,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    
    chatbot_response = response.choices[0].message.content.strip()
    return chatbot_response
invoke_llm("This is a test")

In [295]:
def get_prompt(question, context):
    prompt = f"""Question: {question}
            System: Let's think step by step.
            Context: {context} 
            """
    return prompt
def get_recommendation_prompt(query, context):
    prompt = f"""
        From the given movie listing data, choose a few great movie recommendations.
        User query: {query}
        Context: {context} 
        
        Movie Recommendations:
        1. Movie_name: Movie_overview
        """
    return prompt

In [297]:
idea = "In which movie does the footballer goes completly blind?"
search_response = query_vector_search(idea, prefilter={"year":{"$gt": 1990}}, postfilter={"score": {"$gt":0.8}},topK=10)
premise = "\n".join(list(map(lambda x:x['final'], search_response)))
prompt = get_prompt(idea, premise)
invoke_llm(prompt)

'The movie in which the footballer goes completely blind is "23 Blast."'

In [298]:
question = "I like Christmas movies, any recommendations for movies release after 1990?"
search_response = query_vector_search(question,topK=10)
context = "\n".join(list(map(lambda x:x['final'], search_response)))
print(invoke_llm(get_recommendation_prompt("I like Christmas movies, any recommendations for movies release after 1990?", context)))

1. Happy Christmas: After a breakup with her boyfriend, a young woman moves in with her older brother, his wife, and their 2-year-old son.
2. Almost Christmas: A dysfunctional family gathers together for their first Christmas since their mom died.
3. Finding Christmas: Single mother Ryan's dating life changes when her brother swaps homes with a New York City adman.
4. Christmas Eve: Hilarity, romance, and transcendence prevail after a power outage traps six different groups of New Yorkers inside elevators on Christmas Eve.
5. National Lampoon's Christmas Vacation: The Griswolds prepare for a family seasonal celebration, but things never run smoothly for Clark, his wife Ellen, and their two kids.


In [283]:
print(invoke_llm("In which movie does a footballer go completely blind?"))

The movie you are referring to is "The Damned United" (2009), where the character Brian Clough, a former footballer and manager, goes temporarily blind due to stress and anxiety.


In [284]:
idea = "In which movie does a footballer go completely blind?"
search_response = query_vector_search(idea, prefilter={"year":{"$gt": 1990}}, postfilter={"score": {"$gt":0.8}},topK=10)
premise = "\n".join(list(map(lambda x:x['final'], search_response)))
print(invoke_llm(get_prompt(idea, premise)))

The movie in which a footballer goes completely blind is "23 Blast."


# Semantic Similarity with Langchain

In [330]:
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.embeddings import OpenAIEmbeddings
import json

embedding_model = OpenAIEmbeddings()
vector_search = MongoDBAtlasVectorSearch(output_collection, embedding_model, text_key='final')
fquery = {"year": {"$gt": 1990}}
search_kwargs = {
    "k": 5,
    'filter': fquery,
}
retriever = vector_search.as_retriever(search_kwargs=search_kwargs)
docs = retriever.get_relevant_documents("I like Christmas movies, any recommendations for movies release after 1990?")
for doc in docs:
    foo = {}
    foo['title'] = doc.metadata['title']
    foo['year'] = doc.metadata['year']
    foo['final'] = doc.metadata['text']
    print(json.dumps(foo,indent=1))

{
 "title": "Christmas Vacation '91",
 "year": 1991,
 "final": "Title: Christmas Vacation '91  Genres: Animation,Comedy,FamilyThis coarse bedroom farce takes place at the St. Moritz ski resort over a Christmas vacation. Among the couples whose lives intersect are a widowed artist honeymooning with his second wife, a gay man traveling with his son and his lover (and hiding each from the other), a snobbish couple from Milan who have been forced to share a suite with a pair of crass Romans, etc."
}
{
 "title": "Happy Christmas",
 "year": 2014,
 "final": "Title: Happy Christmas  Genres: Animation,Comedy,FamilyAfter a breakup with her boyfriend, a young woman moves in with her older brother, his wife, and their 2-year-old son."
}
{
 "title": "Almost Christmas",
 "year": 2016,
 "final": "Title: Almost Christmas  Genres: Animation,Comedy,FamilyA dysfunctional family gathers together for their first Christmas since their mom died."
}
{
 "title": "Finding Christmas",
 "year": 2013,
 "final": "T

# Chunking Strategies

In [331]:
!pip install -qU langchain-text-splitters pypdf2

In [333]:
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def get_pdf_data(pdf_docs):
    for pdf in pdf_docs:
        text = ""
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text
text = get_pdf_data(["./Uber-Annual-Report.pdf"])


"Uber \n2021 \nAnnual ReportUber’s Mission\nWe reimagine the way the world moves for the better\nWe are Uber. The go-getters. The kind of people who are relentless about our mission to help people go anywhere and get anything  and earn their way . \nMovement is what we  power . It’s our lifeblood. It runs through our veins. It’s what gets us out of bed each morning. It pushes us to constantly reimagine \nhow we can move better. For you. For all the places you want to go. For all the things you want to get. For all the ways you want to earn. Across the entire \nworld. In real time . At the incredible speed of now.\nOur Values\nOur values reflect who we are and where we’re going. They guide our decision-making, unite and define our culture, and tell a story to the world about \nUber’s corporate purpose.\nDo the right thingPeriod.\nGo get itBring the mindset of a champion.\nOur ambition is what drives us to achieve our mission. How we define a champion mindset \nisn't based on how we perf

In [345]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_text(text)
len(chunks)

797

In [346]:
print("\n--------------")
print("First Chunk:\n"+chunks[0]+ "\n--------------")
print("Second Chunk:\n"+chunks[1]+ "\n--------------")


--------------
First Chunk:
Uber 
2021 
Annual ReportUber’s Mission
We reimagine the way the world moves for the better
We are Uber. The go-getters. The kind of people who are relentless about our mission to help people go anywhere and get anything  and earn their way . 
Movement is what we  power . It’s our lifeblood. It runs through our veins. It’s what gets us out of bed each morning. It pushes us to constantly reimagine 
how we can move better. For you. For all the places you want to go. For all the things you want to get. For all the ways you want to earn. Across the entire 
world. In real time . At the incredible speed of now.
Our Values
Our values reflect who we are and where we’re going. They guide our decision-making, unite and define our culture, and tell a story to the world about 
Uber’s corporate purpose.
Do the right thingPeriod.
Go get itBring the mindset of a champion.
Our ambition is what drives us to achieve our mission. How we define a champion mindset
-------------

In [350]:
!pip install -qU  spacy
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 27.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [353]:
from langchain_text_splitters import SpacyTextSplitter

text_splitter = SpacyTextSplitter(chunk_size=500)

sentence_chunks = text_splitter.split_text(text)
len(sentence_chunks)

/Users/ashwin.gangadhar/anaconda3/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
Created a chunk of size 952, which is longer than the specified 500
Created a chunk of size 509, which is longer than the specified 500
Created a chunk of size 792, which is longer than the specified 500
Created a chunk of size 659, which is longer than the specified 500
Created a chunk of size 626, which is longer than the specified 500
Created a chunk of size 896, which is longer than the specified 500
Created a chunk of size 1024, which is longer than the specified 500
Created a chunk of size 1288, which is longer than the specified 500
Created a chunk of size 1186, which is longer than the specified 500
Created a chunk of size 749, which 

1497

In [354]:
print("\n--------------")
print("First Chunk:\n"+sentence_chunks[0]+ "\n--------------")
print("Second Chunk:\n"+sentence_chunks[1]+ "\n--------------")


--------------
First Chunk:
Uber 
2021 
Annual ReportUber’s Mission
We reimagine the way the world moves for the better
We are Uber.

The go-getters.

The kind of people who are relentless about our mission to help people go anywhere and get anything  and earn their way . 


Movement is what we  power .

It’s our lifeblood.

It runs through our veins.

It’s what gets us out of bed each morning.

It pushes us to constantly reimagine 
how we can move better.

For you.

For all the places you want to go.
--------------
Second Chunk:
It’s our lifeblood.

It runs through our veins.

It’s what gets us out of bed each morning.

It pushes us to constantly reimagine 
how we can move better.

For you.

For all the places you want to go.

For all the things you want to get.

For all the ways you want to earn.

Across the entire 
world.

In real time .

At the incredible speed of now.


Our Values
Our values reflect who we are and where we’re going.
--------------


In [355]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=500)
nltk_chunks = text_splitter.split_text(text)

In [357]:
print("\n--------------")
print("First Chunk:\n"+sentence_chunks[0]+ "\n--------------")
print("Second Chunk:\n"+sentence_chunks[1]+ "\n--------------")


--------------
First Chunk:
Uber 
2021 
Annual ReportUber’s Mission
We reimagine the way the world moves for the better
We are Uber.

The go-getters.

The kind of people who are relentless about our mission to help people go anywhere and get anything  and earn their way . 


Movement is what we  power .

It’s our lifeblood.

It runs through our veins.

It’s what gets us out of bed each morning.

It pushes us to constantly reimagine 
how we can move better.

For you.

For all the places you want to go.
--------------
Second Chunk:
It’s our lifeblood.

It runs through our veins.

It’s what gets us out of bed each morning.

It pushes us to constantly reimagine 
how we can move better.

For you.

For all the places you want to go.

For all the things you want to get.

For all the ways you want to earn.

Across the entire 
world.

In real time .

At the incredible speed of now.


Our Values
Our values reflect who we are and where we’re going.
--------------


In [360]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
markdown_document = "# Foo\n\n    ## Bar\n\nHi this is Jim\n\nHi this is Joe\n\n ### Boo \n\n Hi this is Lance \n\n ## Baz\n\n Hi this is Molly"

headers_to_split_on = [
   ("#", "Header 1"),
   ("##", "Header 2"),
   ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_document)
print("\n--------------")
print("First Chunk:\n"+md_header_splits[0].page_content+ "\n--------------")
print("Second Chunk:\n"+md_header_splits[1].page_content+ "\n--------------")


--------------
First Chunk:
Hi this is Jim  
Hi this is Joe
--------------
Second Chunk:
Hi this is Lance
--------------


In [ ]:
!pip install -qU langchain_experimental

In [363]:

from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings

text_splitter = SemanticChunker(HuggingFaceEmbeddings())

semantic_chunks = text_splitter.split_text(text)
print("\n--------------")
print("First Chunk:\n"+semantic_chunks[0]+ "\n--------------")
print("Second Chunk:\n"+semantic_chunks[1]+ "\n--------------")


--------------
First Chunk:
Uber 
2021 
Annual ReportUber’s Mission
We reimagine the way the world moves for the better
We are Uber. The go-getters.
--------------
Second Chunk:
The kind of people who are relentless about our mission to help people go anywhere and get anything  and earn their way . Movement is what we  power .
--------------


# Advanced RAG

In [1]:
s3_uri= "s3://ashwin-partner-bucket/fashion_dataset.jsonl"

# Sample Dataset with OpenAIEmbedding already included in the dataset

In [2]:
import pandas as pd
import s3fs
import boto3
df = pd.read_json(s3_uri, orient="records", lines=True)
df[:3]

,ageGroup,link,brandName,price,title,gender,subCategory,masterCategory,season,articleType,baseColour,id,openAIVec
0,Adults-Women,http://assets.myntassets.com/v1/images/style/p...,Inc 5,1390.0,Inc. 5 Women Casual White Flats,Women,Shoes,Footwear,Winter,Heels,White,22275,"[-0.016259265699646003, -0.016057870922684, -0..."
1,Adults-Women,http://assets.myntassets.com/v1/images/style/p...,French Connection,3999.0,French Connection Women Black Sling Bag,Women,Bags,Accessories,Summer,Handbags,Black,42874,"[-0.022201561751436002, 0.006381784873631001, ..."
2,Kids-Girls,http://assets.myntassets.com/v1/images/style/p...,Q&Q,625.0,Q&Q Kids Girls White Dial Analog Watch,Girls,Watches,Accessories,Winter,Watches,Pink,49888,"[-0.001154974972115, 0.010144626266777, -7.218..."


# Insert Data to MongoDB Atlas

In [25]:
from pymongo import MongoClient
import certifi
mongo_client = MongoClient(os.environ["MONGODB_CONNECTION_STR"], tlsCAFile=certifi.where())
# Upload documents along with vector embeddings to MongoDB Atlas Collection
col = mongo_client["search"]["catalog_final_myn"]
col.insert_many(df.to_dict(orient="records"))

In [21]:

from langchain_core.output_parsers import JsonOutputParser # type: ignore
from langchain_core.prompts import PromptTemplate # type: ignore
from langchain_core.pydantic_v1 import BaseModel, Field # type: ignore
from langchain_openai import ChatOpenAI # type: ignore

from langchain_openai.embeddings import OpenAIEmbeddings # type: ignore
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch # type: ignore


from pymongo import MongoClient # type: ignore
from typing import List
from itertools import chain
import certifi # type: ignore
import os
from dotenv import load_dotenv # type: ignore

load_dotenv()

from functools import lru_cache

@lru_cache
def get_openai_emb_transformers():
    """
    Returns an instance of OpenAIEmbeddings for OpenAI transformer models.
    
    This function creates and returns an instance of the OpenAIEmbeddings class,
    which provides access to OpenAI transformer models for natural language processing.
    The instance is cached using the lru_cache decorator for efficient reuse.
    
    Returns:
        embeddings (OpenAIEmbeddings): An instance of the OpenAIEmbeddings class.
    """
    embeddings = OpenAIEmbeddings()
    return embeddings

@lru_cache
def get_vector_store():
    """
    Retrieves the vector store for MongoDB Atlas.

    Returns:
        MongoDBAtlasVectorSearch: The vector store object.
    """
    vs = MongoDBAtlasVectorSearch(collection=col, embedding=get_openai_emb_transformers(), index_name="vector_index_openAi_cosine", embedding_key="openAIVec", text_key="title")
    return vs

@lru_cache(10)
def get_conversation_chain_conv():
    """
    Retrieves a conversation chain model for chat conversations.

    Returns:
        ChatOpenAI: The conversation chain model for chat conversations.
    """
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2, max_tokens=2048)
    # chain = ConversationChain(llm=llm, memory=ConversationBufferWindowMemory(k=5))
    return llm


# Define your desired data structure.
class ProductRecoStatus(BaseModel):
    """
    Represents the status of product recommendations.

    Attributes:
        relevancy_status (bool): Product recommendation status conditioned on the context of the input query.
                                 True if the query is related to purchasing fashion clothing and/or accessories.
                                 False otherwise.
        recommendations (List[str]): List of recommended product titles based on the input query context and
                                     if the relevancy_status is True.
    """
    relevancy_status: bool = Field(description="Product recommendation status is conditioned on the fact if the context of input query is to purchase a fashion clothing and or fashion accessories.")
    recommendations: List[str] = Field(description="list of recommended product titles based on the input query context and if recommendation_status is true.")


class Product(BaseModel):
    """
    Represents a product.

    Attributes:
        title (str): Title of the product.
        baseColour (List[str]): List of base colours of the product.
        gender (List[str]): List of genders the product is targeted for.
        articleType (str): Type of the article.
        mfg_brand_name (str): Manufacturer or brand name of the product.
    """
    title: str = Field(description="Title of the product.")
    baseColour: List[str] = Field(description="List of base colours of the product.")
    gender: List[str] = Field(description="List of genders the product is targeted for.")
    articleType: str = Field(description="Type of the article.")
    mfg_brand_name: str = Field(description="Manufacturer or brand name of the product.")


class Recommendations(BaseModel):
    """
    Represents a set of recommendations for products and a message to the user.

    Attributes:
        products (List[Product]): List of recommended products.
        message (str): Message to the user and context of the chat history summary.
    """
    products: List[Product] = Field(description="List of recommended products.")
    message: str = Field(description="Message to the user and context of the chat history summary.")


reco_status_parser = JsonOutputParser(pydantic_object=ProductRecoStatus)

reco_status_prompt = PromptTemplate(
    template="You are AI assistant tasked at identifying if there is a product purchase intent in the query and providing suitable fashion recommendations.\n{format_instructions}\n{query}\n\
        #Chat History Summary: {chat_history}\n\nBased on the context of the query, please provide the relevancy status and list of recommended products.",
    input_variables=["query", "chat_history"],
    partial_variables={"format_instructions": reco_status_parser.get_format_instructions()},
)

reco_parser = JsonOutputParser(pydantic_object=Recommendations)
reco_prompt = PromptTemplate(
    input_variables=["question", "recommendations", "chat_history"],
    partial_variables={"format_instructions": reco_parser.get_format_instructions()},
    template="\n User query:{question} \n Chat Summary: {chat_history} \n Rank and suggest me suitable products for creating grouped product recommendations given all product recommendations below feature atleast one product for each articleType \n {recommendations} \n show output in {format_instructions} for top 10 products"
)


def get_product_reco_status(query: str, chat_history: List[str] = []):
    """
    Retrieves the recommendation status for a product based on the given query and chat history.

    Args:
        query (str): The query to be used for retrieving the recommendation status.
        chat_history (List[str]): The chat history containing previous conversations.

    Returns:
        The response containing the recommendation status.
    """
    llm = get_conversation_chain_conv()
    chain = reco_status_prompt | llm | reco_status_parser
    resp = chain.invoke({"query": query, "chat_history": chat_history})
    return resp

def get_sorted_results(product_recommendations):
    all_titles = [rec['title'] for rec in product_recommendations['products']]
    results = list(col.find({"title": {"$in":all_titles}}, {"_id": 0 , "id":1, "title": 1, "price": 1, "baseColour": 1, "articleType": 1, "gender": 1, "link" : 1, "mfg_brand_name": 1}))
    sorted_results = []
    for title in all_titles:
        for result in results:
            if result['title'] == title:
                sorted_results.append(result)
                break
    return sorted_results

def get_product_recommendations(query: str, reco_queries: List[str], chat_history: List[str]=[]):
    """
    Retrieves product recommendations based on the given query and chat history.

    Args:
        query (str): The query string for the recommendation.
        chat_history (List[str]): The list of previous chat messages.
        filter_query (dict): The filter query to apply during the recommendation retrieval.
        reco_queries (List[str]): The list of recommendation queries.

    Returns:
        dict: The response containing the recommendations.

    """
    vectorstore = get_vector_store()
    retr = vectorstore.as_retriever(search_kwargs={"k": 10})
    all_recommendations = list(chain(*retr.batch(reco_queries)))
    llm = get_conversation_chain_conv()
    llm_chain = reco_prompt | llm | reco_parser
    resp = llm_chain.invoke({"question": query, "chat_history": chat_history, "recommendations": [v.page_content for v in all_recommendations]})
    resp = get_sorted_results(resp)
    return resp

In [30]:
query = "Can you suggest me some Casual dresses for date occassion with my boy friend"
status = get_product_reco_status(query)
print(status)
print(get_product_recommendations(query, reco_queries=status["recommendations"], chat_history=[]))
print(get_conversation_chain_conv().invoke(query).content)

{'relevancy_status': True, 'recommendations': ['Floral Wrap Dress', 'Off-Shoulder Maxi Dress', 'Lace Fit and Flare Dress', 'Ruffled Hem Shift Dress', 'Denim Shirt Dress']}
[{'link': 'http://assets.myntassets.com/v1/images/style/properties/ebb8a69f6e56cf47f9fefd3ac23cfe03_images.jpg', 'price': 690.0, 'title': 'Femella Women Floral Red Dress', 'gender': 'Women', 'mfg_brand_name': 'Femella', 'articleType': 'Dresses', 'baseColour': 'Red', 'id': '39217'}, {'link': 'http://assets.myntassets.com/v1/images/style/properties/730a44ed829f9310beb97f583a960262_images.jpg', 'price': 3800.0, 'title': 'Forever New Women Floral Purple Dress', 'gender': 'Women', 'mfg_brand_name': 'Forever New', 'articleType': 'Dresses', 'baseColour': 'Purple', 'id': '8484'}, {'link': 'http://assets.myntassets.com/v1/images/style/properties/4e056a5231b2f0f427e2b9f2d45a2662_images.jpg', 'price': 1849.0, 'title': 'Mineral Women Floral Orange Dress', 'gender': 'Women', 'mfg_brand_name': 'Mineral', 'articleType': 'Dresses', 

In [29]:
query = "Where should I take my boy friend for date" 
status = get_product_reco_status(query) 
print(status) 
print(get_conversation_chain_conv().invoke(query).content) 

{'relevancy_status': False, 'recommendations': []}
There are many great options for a date with your boyfriend, depending on your interests and preferences. Some ideas could include:

1. A romantic dinner at a nice restaurant
2. A picnic in the park or at the beach
3. A movie night at home with homemade popcorn and snacks
4. A hike or nature walk followed by a picnic
5. A visit to a local museum or art gallery
6. A cooking class or wine tasting experience
7. A day trip to a nearby city or town to explore and try new restaurants
8. A couples massage or spa day
9. A fun activity like mini golf, bowling, or go-kart racing
10. A concert or live music event

Ultimately, the best date idea is one that you both will enjoy and that allows you to spend quality time together. Consider your boyfriend's interests and preferences when planning the date to ensure it is a memorable and enjoyable experience for both of you.


There are many great options for a date with your boyfriend, depending on your interests and preferences. Some ideas could include: 
 
1. A romantic dinner at a nice restaurant 
2. A picnic in the park or at the beach 
3. A movie night at home or at the cinema 
4. A hike or nature walk 
5. A visit to a museum or art gallery 
6. A cooking class or wine tasting 
7. A concert or live music event 
8. A day trip to a nearby city or town 
9. A couples massage or spa day 
10. A fun activity like mini golf, bowling, or go-kart racing 
 
Ultimately, the best date idea is one that you both will enjoy and that allows you to spend quality time together. Consider what you both like to do and choose an activity that will create lasting memories. 